In [1]:
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import pandas as pd
plt.rcParams['figure.figsize'] = [11,7]

## 0. Assumptions and Constants

In [2]:
#Molar Masses (dict):
element_mw = {'li':6.94,'ni':58.69,'mn':54.94,'co':58.93,'al':26.98,'o':16.00}

In [3]:
#Dimensions - 18650 cells
width = 5.5 #cm
length = 110 #cm
one_sided_area = width*length #cm2, area ~ 600cm2 https://www.quora.com/What-is-the-surface-area-of-an-anode-in-a-single-18650-lithium-ion-battery

#Echem characteristics
# porosity = .3 #not sure if useful
cat_loading = 25 #mg/cm2, this is active loading only. Everything that is per gram is on active material basis.
avg_voltage = 3.6 #V
an_capacity = 360 #mAh/g, graphite
np_ratio = 1.02 #unitless, typical for commercial cells

In [4]:
#Capacities
nmc111_cap = 160 #mah/g active
nmc523_cap = 165 #mah/g
nmc622_cap = 170 #mah/g
nmc811_cap = 180 #mah/g
ni100_cap = 190 #mah/g
nca801505_cap = 170 #mah/g
nca880903_cap = 180 #mah/g
nmc111mod_cap = 190 #mah/g

This is a reasonable value for anode loading.

In [5]:
#Let's now calculate the total cell energy:
def get_cell_energy(cat_capacity, cat_loading,one_sided_area=one_sided_area,avg_voltage=avg_voltage):
    '''
    Returns energy of one 18650 cell (in Wh) given a cathode specific capacity (mAh/g), cathode loading, 
    electrode area (cm^2), and average discharge voltage.
    '''
    two_sided_area = 2*one_sided_area #cm2
    cell_capacity = cat_capacity*((cat_loading/1000)*two_sided_area)/1000 #Ah
    cell_energy = cell_capacity*avg_voltage #Wh
    return cell_energy #in Wh

In [6]:
get_cell_energy(ni100_cap, cat_loading)

20.691

In [7]:
nmc111_energy = (get_cell_energy(nmc111_cap, cat_loading,one_sided_area,avg_voltage))
nmc523_energy = (get_cell_energy(nmc523_cap, cat_loading,one_sided_area,avg_voltage))
nmc622_energy = (get_cell_energy(nmc622_cap, cat_loading,one_sided_area,avg_voltage))
nmc811_energy = (get_cell_energy(nmc811_cap, cat_loading,one_sided_area,avg_voltage))
ni100_energy = (get_cell_energy(ni100_cap, cat_loading,one_sided_area,avg_voltage))
nca801505_energy = (get_cell_energy(nca801505_cap, cat_loading,one_sided_area,avg_voltage))
nca880903_energy = (get_cell_energy(nca880903_cap, cat_loading,one_sided_area,avg_voltage))
nmc111mod_energy = (get_cell_energy(nmc111mod_cap, cat_loading,one_sided_area,avg_voltage))

According to source, Panasonic had 13.6Wh cells in 2013. If we assume a 3% rate of capacity improvement over the past 6 years:

In [8]:
pana_cap_2019 = 13.6*(1.03)**6
print(pana_cap_2019)

16.2391112327944


With 16.3 Wh, we see that our modeled cell energies are in the same range as the estimated Panasonic capacity.

## 1. Cathode Cost
The primary cost we're looking at is cathode cost. We will assume that all cells use the same cathode mass, and we will calculate the materials cost accordingly.
#### Part 1: Material content per battery

In [9]:
#Total cathode, binder, and conductor masses
cat_mass_per_cell = cat_loading*.001 * 2*one_sided_area #g
cblack_mass = 1/98 * cat_mass_per_cell
pvdf_mass = 1/98 * cat_mass_per_cell
print('Cathode active material mass per cell = '+ str(cat_mass_per_cell) + ' g')

Cathode active material mass per cell = 30.25 g


In [10]:
#Aluminum foil masses:
al_density = 2.7 #g/cm3
alfoil_thickness = 20 #um
alfoil_mass = one_sided_area*(alfoil_thickness*1e-4)*al_density

In [11]:
# To get metal contents for each cathode composition, we'll start by representing cathode chemical formulas as dicts.
nmc111_formula = {'li':1,'ni':.333,'mn':.333,'co':.333,'al':0,'o':2}
nmc523_formula = {'li':1,'ni':.5,'mn':.2,'co':.3,'al':0,'o':2}
nmc622_formula = {'li':1,'ni':.6,'mn':.2,'co':.2,'al':0,'o':2}
nmc811_formula = {'li':1,'ni':.8,'mn':.1,'co':.1,'al':0,'o':2}
ni100_formula = {'li':1,'ni':1,'mn':0,'co':0,'al':0,'o':2}
nca801505_formula = {'li':1,'ni':.8,'mn':0,'co':.15,'al':.05,'o':2}
nca880903_formula = {'li':1,'ni':.88,'mn':0,'co':.09,'al':.03,'o':2}
nmc111mod_formula = nmc111_formula

In [12]:
# Calculating cells per kWh:
def get_n_cells_per_kwh(cell_energy):
    return 1000/cell_energy

#Function for calculating molar mass
def get_molar_mass(formula):
    element_mw = {'li':6.94,'ni':58.69,'mn':54.94,'co':58.93,'al':26.98,'o':16.00}
    molar_mass = 0
    for i in formula:
        molar_mass += formula[i]*element_mw[i]
    return molar_mass

In [13]:
#Now that we have molar masses, we can make a function to get total material mass per kWh:
def get_material_mass_kwh(formula,cell_energy,cat_mass_per_cell=cat_mass_per_cell):
    '''
    Returns mass (in g) for each element of the cathode as a dict. 
    Inputs are chemical formula (as a dict), cell_energy (in Wh), and mass of cathode active material per cell.
    '''
    molar_mass = get_molar_mass(formula)
    kwh_material_mass = {}
    for i in formula:
        cell_material_mass = cat_mass_per_cell*(formula[i]*element_mw[i])/get_molar_mass(formula)
        kwh_material_mass[i] = cell_material_mass * get_n_cells_per_kwh(cell_energy)
    return kwh_material_mass #in g

In [14]:
get_material_mass_kwh(nmc111_formula,nmc111_energy)

{'li': 124.98237712464568,
 'ni': 351.9635205442848,
 'mn': 329.47479670647476,
 'co': 353.4027988699046,
 'al': 0.0,
 'o': 576.2876178658014}

Therefore, our model tells us that 1kWh of NMC111 cells will contain 353.4g of cobalt. This is comparable to estimates from other sources (https://techcrunch.com/2017/01/01/no-cobalt-no-tesla/ says it is 360g Co per kWh.)

#### Part 2: Adding in price data:
Note: these figures are inherently very uncertain because contracts negotiated by large companies will differ substantially from open market prices. Additionally, the raw inputs to cathode syntheses are generally in the form of hydroxides (not metals), which adds additional process costs.

Prices are from Bloomberg New Energy Finance for Li and Co, and from London Metals Exchange for the others. BNEF is likely more accurate because it takes into account more characteristics required for battery grade materials.

In [15]:
#Carbon black and PVDF
cblack_price = 600 #USD/t, Zhengzhou Qiangjin Science And Technology Trading Co., Ltd.
pvdf_price = 18 #USD/kg, Shenyang East Chemical Science-Tech Co., Ltd.
alfoil_price = 2200 #USD/t (note this is a separate cost from al in the battery)

In [16]:
#Metals prices
oh_process_cost = 200 #cost to convert metal to metal hydroxide, assuming constant
ni_price = 16305 + oh_process_cost #USD/metric ton (t)
mn_price = 1610 + oh_process_cost #USD/t, from Shanghai Metals Market https://price.metal.com/Manganese
co_price = 50000 + oh_process_cost #USD/t
al_price = 1817.5 + oh_process_cost #USD/t

In [17]:
#Lithium price will have to come from LiOH-H2O price because metallic lithium isn't traded. 
#Let's normalize the LiOH-H2O price to be on a per g Li basis to match the others.
#Note that this is not subject to hydroxide process cost shown above because it comes as hydroxide.
lioh_h2o_price = 15000 #USD/t
lioh_h2o_mw = 41.96 #g/mol
li_mw = 6.94

#To get 1t of Li metal, you'll need 41.96/6.94 tons of LiOH. Therfore Li price = lioh-h2o price * 41.96/6.94
mw_ratio = 41.96/6.94
li_price = lioh_h2o_price * mw_ratio
print(li_price)

90691.64265129683


In [18]:
metals_prices = {'li':li_price,'ni':ni_price,'mn':mn_price,'co':co_price,'al':al_price,'o':0}

In [19]:
#Finally, we add the cost for turning the raw hydroxide powders into the cathode powder. 
#This process involves baking around 500-700C in oxygen. Estimates are around 50-200 USD/ton active material. (source?)
synthesis_process_cost = 150 #USD per ton active material

In [20]:
#Now, for every cathode composition, we can easily calculate the total materials cost:
def get_cathode_cost(formula,cell_energy,cat_mass_per_cell=cat_mass_per_cell,metals_prices=metals_prices,
                     synthesis_process_cost=synthesis_process_cost):
    '''
    Calculates total cost per kWh of a given cathode chemistry, including materials and processing costs.
    '''
    
    #Metals cost
    #Get masses of each metal in the cathode:
    material_mass = get_material_mass_kwh(formula,cell_energy,cat_mass_per_cell)
    #Then loop through each element in cathode:
    cat_metals_cost = 0
    for i in material_mass:
        cat_metals_cost += material_mass[i]/1e6 * metals_prices[i]
        
    #Cathode other materials cost (binder, carbon black, al foil):
    cat_other_cost = (cblack_mass/1e6)*cblack_price + (pvdf_mass/1e3)*pvdf_price + (alfoil_mass/1e6)*alfoil_price
        
    #Processing cost:
    cat_mass_per_kwh = cat_mass_per_cell*get_n_cells_per_kwh(cell_energy) #g
    cat_process_cost = (cat_mass_per_kwh/1e6)*synthesis_process_cost
    
    return {'cat_metals':cat_metals_cost,'cat_other':cat_other_cost,'cat_process':cat_process_cost}

In [21]:
[nmc111_cost,nmc523_cost,nmc622_cost,nmc811_cost,ni100_cost] = [get_cathode_cost(nmc111_formula,nmc111_energy),
                                                                get_cathode_cost(nmc523_formula,nmc523_energy),
                                                                get_cathode_cost(nmc622_formula,nmc622_energy),
                                                                get_cathode_cost(nmc811_formula,nmc811_energy),
                                                                get_cathode_cost(ni100_formula,ni100_energy)]

In [22]:
nmc111_cost

{'cat_metals': 35.48118487578933,
 'cat_other': 0.012928726530612243,
 'cat_process': 0.26041666666666663}

In [45]:
get_n_cells_per_kwh(nmc111_energy)

57.3921028466483

Want to think about some ways to structure this data more effectively for analysis - there should eventually be a pie chart showing the breakdown of cell cost between various cathode materials, processing costs, anode costs, cell can, etc.

## 2. Anode Cost

First, we want to get the total mass of active anode material per cell.

In [23]:
# Now we use our assumptions to calculate the anode loading and total cell energy
#getting mAh/cm2:
def get_anode_loading(cat_capacity, cat_loading = cat_loading, an_capacity = an_capacity, np_ratio = np_ratio):
    cat_area_capacity = cat_capacity * (.001*cat_loading) #mAh/cm2
    an_area_capacity = cat_area_capacity*np_ratio #mAh/cm2
    an_loading = an_area_capacity/an_capacity*1000 #mg/cm2
    return an_loading

# getting total anode mass: 
def get_graphite_mass_kwh(cat_capacity, cell_energy,cat_loading = cat_loading):
    an_loading = get_anode_loading(cat_capacity, cat_loading = cat_loading)
    gr_mass_per_cell = an_loading*.001 * 2*one_sided_area #g
    gr_mass_per_kwh = gr_mass_per_cell * get_n_cells_per_kwh(cell_energy)
    return gr_mass_per_kwh #graphite mass in g

def get_cu_mass_kwh(cat_capacity, cell_energy, one_sided_area, cu_thickness = 10):
    #cu_thickness in microns
    cu_density = 8.96 #g/cm3
    cu_mass_per_cell = one_sided_area*(cu_thickness*1e-4)*cu_density #g
    cu_mass_per_kwh = cu_mass_per_cell*get_n_cells_per_kwh(cell_energy) #g
    return cu_mass_per_kwh

In [24]:
get_graphite_mass_kwh(nmc111_cap,nmc111_energy)

787.0370370370371

We'll just assume a 98:2 of active material to binder. Assume SBR is only binder.

In [25]:
graphite_price = 2000 #USD/t, Guangzhou Billion Peak Chemical Technology Co., Ltd.
sbr_price = 1500 #USD/t, sbr will be assumed as only binder
cu_price = 5951*2 #USD/t, LME, multiplied by 2 because Cu in micron foil form is much more expensive.
an_prices = {'graphite': graphite_price,'sbr':sbr_price,'cu':cu_price}

In [26]:
def get_anode_cost(cat_capacity, cell_energy, cat_loading=cat_loading, an_prices=an_prices):
    gr_cost = an_prices['graphite'] * get_graphite_mass_kwh(cat_capacity, cell_energy)/1e6
    sbr_cost = an_prices['sbr'] * (2/98)*get_graphite_mass_kwh(cat_capacity, cell_energy)/1e6
    cu_cost = an_prices['cu'] * get_cu_mass_kwh(cat_capacity, cell_energy, one_sided_area, cu_thickness = 10)/1e6
    return gr_cost+sbr_cost+cu_cost

In [27]:
get_anode_cost(nmc111_cap,nmc111_energy)

5.301011489040061

## 3. Other Cell Materials

Electrolyte: just assume a flat cost of 0.2 USD per cell. Reasonable based on BNEF estimates.

In [28]:
def get_electrolyte_cost(cell_energy,el_cost_per_cell = 0.2):
    '''Returns cost in USD/kWh for electrolyte.'''
    return get_n_cells_per_kwh(cell_energy) * el_cost_per_cell

Cell Can: Assume flat cost of 0.2 USD per cell also.

In [29]:
def get_can_cost(cell_energy,can_cost_per_cell = 0.2):
    '''Returns cost in USD/kWh for can.'''
    return get_n_cells_per_kwh(cell_energy) * can_cost_per_cell

Separator: Assume flat cost of 0.05 USD/cell

In [30]:
def get_sep_cost(cell_energy,sep_cost_per_cell = 0.05):
    '''Returns cost in USD/kWh for separator.'''
    return get_n_cells_per_kwh(cell_energy) * sep_cost_per_cell

## 4. Manufacturing Cost
This includes electrode calendering, winding, wetting, tabbing, crimping, etc. Because this requires a large number of steps, this number is inherently very uncertain. We'll just assume a flat cost of 0.5 USD per cell.

In [31]:
def get_cell_manufacturing_cost(cell_energy,mfg_cost_per_cell = 0.5):
    '''Returns cost in USD/kWh for cell manufacturing.'''
    return get_n_cells_per_kwh(cell_energy) * mfg_cost_per_cell

Another significant cost is depreciation of capital. Cell manufacturing requires significant capital investment, which must be replaced over time. We will also assume this cost is on a per cell basis. 

This number has a very high degree of uncertainty due to uncertainty in equipment cost and in number of cells that can be produced. Reasoable estimates put the figure at around 0.3 USD/cell.

In [32]:
def get_cell_depreciation_cost(cell_energy,dep_cost_per_cell):
    '''Returns cost in USD/kWh for cell manufacturing.
    Default = .3 USD/cell'''
    return get_n_cells_per_kwh(cell_energy) * dep_cost_per_cell

## 5. Manufacturing Yield and Contingency

During manufacturing, some cells will inevitably fail due to manufacturing defects. Siginificant effort has gone toward improving manufacturing yields, but they still add a great deal of cost.

We will assume a manufacturing yield rate of 95% on a cell basis. This will cause the cost per cell to be multiplied by (100/98).

We will also add a 15% contingency cost for other miscellaneous costs like labor and transportation.

In [33]:
def get_adjustment_coeff(yield_rate,contingency):
    '''Get adjustment coefficient for costs. Inputs are yield rate and contingency rate in fraction form.
    Default yield rate = .95 and contingency is .15 (15%).
    Multiply all final costs by this adjustment coeff to get actaul cost.'''
    return (1/yield_rate)*(1+contingency)

## Pack Costs
Assuming flat cost of .6 USD/cell.

In [34]:
def get_pack_cost(cell_energy,pack_cost_per_cell = 0.6):
    '''Returns cost in USD/kWh for cell manufacturing.'''
    return get_n_cells_per_kwh(cell_energy) * pack_cost_per_cell

## 6. Full Model
Now that we have a method for calculating each cost, we can sum the results to get our total cost estimate and a cost breakdown.

In [35]:
def get_full_cost_kwh(cat_capacity, cell_energy, formula):
    
    #Initialize dictionary:
    full_cost = {}
    #Cathode cost section:
    cathode_cost = get_cathode_cost(formula,cell_energy,cat_mass_per_cell=cat_mass_per_cell,metals_prices=metals_prices,
                     synthesis_process_cost=synthesis_process_cost)
    full_cost['cat_metals'] = cathode_cost['cat_metals']
    full_cost['cat_other'] = cathode_cost['cat_other']
    full_cost['cat_process'] = cathode_cost['cat_process']
    
    #Anode:
    full_cost['anode'] = get_anode_cost(cat_capacity, cell_energy, cat_loading=cat_loading, an_prices=an_prices)
    
    #Other Cell Materials:
    full_cost['other_materials'] = (get_electrolyte_cost(cell_energy,el_cost_per_cell = 0.2) + 
                                    get_can_cost(cell_energy,can_cost_per_cell = 0.2) + 
                                    get_sep_cost(cell_energy,sep_cost_per_cell = 0.05))
    
    #Processing and Depreciation Cost:
    full_cost['cell_mfg'] = get_cell_manufacturing_cost(cell_energy,mfg_cost_per_cell = 0.5)
    full_cost['depreciation'] = get_cell_depreciation_cost(cell_energy,dep_cost_per_cell = 0.3)
    
    #Yield and contingency
    adjustment_coeff = get_adjustment_coeff(yield_rate = .95, contingency = .15)
    
    #Using the adjustment coeff:
    for i in full_cost:
        full_cost[i] = adjustment_coeff*full_cost[i]
        
    #Pack Cost:
    full_cost['pack'] = get_pack_cost(cell_energy,pack_cost_per_cell = 0.6)
    
    print("Full pack level cost = " + str(get_total_cost(full_cost)) + " USD/kWh.")
    
    return full_cost

def get_total_cost(full_cost):
    total_cost = 0
    for i in full_cost:
        total_cost += full_cost[i]
    return total_cost

In [36]:
get_full_cost_kwh(nmc111_cap, nmc111_energy, nmc111_formula)

Full pack level cost = 170.97738893302852 USD/kWh.


{'cat_metals': 42.950908007534444,
 'cat_other': 0.015650563694951658,
 'cat_process': 0.31524122807017535,
 'anode': 6.417013907785336,
 'other_materials': 31.263592866463675,
 'cell_mfg': 34.73732540718186,
 'depreciation': 20.842395244309113,
 'pack': 34.43526170798898}

In [37]:
get_full_cost_kwh(nmc523_cap, nmc523_energy, nmc523_formula)

Full pack level cost = 166.68962575403995 USD/kWh.


{'cat_metals': 42.4836359527061,
 'cat_other': 0.015650563694951658,
 'cat_process': 0.3056884635832004,
 'anode': 6.281183888292159,
 'other_materials': 30.316211264449628,
 'cell_mfg': 33.68467918272181,
 'depreciation': 20.21080750963308,
 'pack': 33.39176892895901}

In [38]:
get_full_cost_kwh(nmc622_cap, nmc622_energy, nmc622_formula)

Full pack level cost = 157.79442078812235 USD/kWh.


{'cat_metals': 37.184187338939374,
 'cat_other': 0.015650563694951658,
 'cat_process': 0.2966976264189885,
 'anode': 6.153343869945637,
 'other_materials': 29.42455799196581,
 'cell_mfg': 32.69395332440645,
 'depreciation': 19.61637199464387,
 'pack': 32.40965807810727}

In [39]:
get_full_cost_kwh(nmc811_cap, nmc811_energy, nmc811_formula)

Full pack level cost = 146.85819695847115 USD/kWh.


{'cat_metals': 32.84018348823134,
 'cat_other': 0.015650563694951658,
 'cat_process': 0.280214424951267,
 'anode': 5.918970502977019,
 'other_materials': 27.78986032574549,
 'cell_mfg': 30.877622584161653,
 'depreciation': 18.52657355049699,
 'pack': 30.609121518212426}

In [40]:
get_full_cost_kwh(ni100_cap, ni100_energy, ni100_formula)

Full pack level cost = 137.0886737213488 USD/kWh.


{'cat_metals': 28.968962337531853,
 'cat_other': 0.015650563694951658,
 'cat_process': 0.26546629732225296,
 'anode': 5.709268016741938,
 'other_materials': 26.327236098074675,
 'cell_mfg': 29.252484553416306,
 'depreciation': 17.551490732049782,
 'pack': 28.99811512251704}

In [41]:
get_full_cost_kwh(nmc111mod_cap, nmc111mod_energy, nmc111mod_formula)

Full pack level cost = 144.28889707437227 USD/kWh.


{'cat_metals': 36.169185690555324,
 'cat_other': 0.015650563694951658,
 'cat_process': 0.26546629732225296,
 'anode': 5.709268016741938,
 'other_materials': 26.327236098074675,
 'cell_mfg': 29.252484553416306,
 'depreciation': 17.551490732049782,
 'pack': 28.99811512251704}

In [42]:
get_full_cost_kwh(nca801505_cap, nca801505_energy, nca801505_formula)

Full pack level cost = 158.71231959751788 USD/kWh.


{'cat_metals': 38.10208614833491,
 'cat_other': 0.015650563694951658,
 'cat_process': 0.2966976264189885,
 'anode': 6.153343869945637,
 'other_materials': 29.42455799196581,
 'cell_mfg': 32.69395332440645,
 'depreciation': 19.61637199464387,
 'pack': 32.40965807810727}

In [43]:
get_full_cost_kwh(nca880903_cap, nca880903_energy, nca880903_formula)

Full pack level cost = 147.81974176907568 USD/kWh.


{'cat_metals': 33.80172829883588,
 'cat_other': 0.015650563694951658,
 'cat_process': 0.280214424951267,
 'anode': 5.918970502977019,
 'other_materials': 27.78986032574549,
 'cell_mfg': 30.877622584161653,
 'depreciation': 18.52657355049699,
 'pack': 30.609121518212426}

In [44]:
ni100_energy

20.691